In [39]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook
import bs4
import re
import os

In [2]:
%load_ext autoreload
%autoreload 2

from sofa_score_scrap import *

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
    "upgrade-insecure-requests": '1',
    "Connection": "close",    
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "max-age=0"
}

### Players Stats 

In [11]:
# s.get('https://www.sofascore.cddom/pt/atletico-mineiro-internacional/qOsCO', headers={'x-test2': 'true'})
resp = requests.get("https://www.sofascore.com/event/7724681/statistics/players/json?_=154328071", headers=headers)

In [12]:
players_df = pd.DataFrame(resp.json()["players"])

In [13]:
get_per_player_data(players_df)

,name,slug,shortName,id,national,team_name,team_id,goals,goalAssist,totalTackle,...,totalCross,totalLongBalls,dispossessed,wasFouled,fouls,saves,punches,runsOut,goodHighClaim,team
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,Juan Cazares,atletico-mineiro,J. Cazares,158411,False,Atlético Mineiro,1977,1,1,0,...,8 (2),2 (2),2,1,2,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Andrés D'Alessandro,internacional,A. D'Alessandro,17639,False,Internacional,1966,1,,1,...,12 (3),5 (2),3,3,1,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Víctor Cuesta,internacional,V. Cuesta,587148,False,Internacional,1966,,,1,...,0 (0),11 (7),0,3,2,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,David Terans,atletico-mineiro,D. Terans,586662,False,Atlético Mineiro,1977,1,,1,...,1 (0),2 (1),0,0,1,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Nico López,internacional,N. López,255881,False,Internacional,1966,,,0,...,5 (2),7 (4),1,1,1,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Patric,atletico-mineiro,Patric,78853,False,Atlético Mineiro,1977,,,4,...,2 (0),8 (1),2,1,1,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Luan,atletico-mineiro,Luan,243427,False,Atlético Mineiro,1977,,,4,...,2 (0),1 (0),0,1,4,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Emerson Santos,internacional,E. Santos,844015,False,Internacional,1966,,,2,...,0 (0),4 (2),1,0,0,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Fabiano,internacional,F. Leismann,551304,False,Internacional,1966,,,2,...,6 (0),5 (2),0,0,2,NaN,NaN,NaN,NaN,INT


### Game Stats

In [14]:
resp = requests.get("https://www.sofascore.com/event/7724681/general/json", headers=headers)

In [16]:
pd.DataFrame(game_statistics(resp, players_df), index=[0]).set_index("game")

,team_away,team_home,data,hora,away_score,home_score,result,Ball possession_home,Ball possession_away,Total shots_home,...,form_minute_82,form_minute_83,form_minute_84,form_minute_85,form_minute_86,form_minute_87,form_minute_88,form_minute_89,form_minute_90,form_minute_90.5
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,CAM,INT,2018-11-21,19:30:00,2,1,-1,54%,46%,15,...,22,73,41,24,1,1,63,29,28,-49


### Pega links dos jogos históricos

#### Pega os IDs das temporadas

In [4]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325")
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
c = bs4.BeautifulSoup(comp.content)

In [5]:
temporadas = c.find_all("a", attrs={"class":"pointer js-uniqueTournament-page-load-season"})

id_temporada = [(a.text.strip(), a.attrs["data-season-id"]) for a in temporadas]
id_temporada

[('19/20', '23776'),
 ('18/19', '17359'),
 ('17/18', '13380'),
 ('16/17', '11733'),
 ('15/16', '10356'),
 ('14/15', '8186'),
 ('13/14', '6311'),
 ('12/13', '4710'),
 ('11/12', '3391'),
 ('10/11', '2746'),
 ('09/10', '2139'),
 ('08/09', '1544'),
 ('07/08', '581'),
 ('06/07', '4'),
 ('05/06', '3'),
 ('04/05', '2'),
 ('03/04', '1'),
 ('02/03', '46'),
 ('01/02', '47'),
 ('00/01', '48'),
 ('99/00', '49'),
 ('98/99', '50'),
 ('97/98', '51'),
 ('92/93', '13043')]

In [23]:
r = bs4.BeautifulSoup(resp.text)

In [24]:
r = resp.json()

In [25]:
teams = set()

for x in r["roundMatches"]["tournaments"]:
    for y in x["events"]:
        teams.add(y["homeTeam"]["name"])
        teams.add(y["awayTeam"]["name"])

teams

KeyError: 'roundMatches'

#### Dê/Para de Siglas

In [91]:
de_para_siglas = pd.read_excel("de_para_siglas_epl.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

## Pega Links dos jogos por Rodadas

#### Página Inicial

In [36]:
s = requests.Session()
s.get(comp.url, headers=headers)

<Response [200]>

#### Para cada temporada

In [112]:
id_temporada[:17]

[('19/20', '23776'),
 ('18/19', '17359'),
 ('17/18', '13380'),
 ('16/17', '11733'),
 ('15/16', '10356'),
 ('14/15', '8186'),
 ('13/14', '6311'),
 ('12/13', '4710'),
 ('11/12', '3391'),
 ('10/11', '2746'),
 ('09/10', '2139'),
 ('08/09', '1544'),
 ('07/08', '581'),
 ('06/07', '4'),
 ('05/06', '3'),
 ('04/05', '2'),
 ('03/04', '1')]

In [116]:
games = resp.json()['roundMatches']["tournaments"][0]['events']

In [121]:
for temp in id_temporada[:17]:
    ret = []
    
    for rodada in tqdm_notebook(range(1, 39)):
        # print(temp[0] + ": " + str(rodada), end="\r")
        resp = s.get("https://www.sofascore.com/u-tournament/17/season/"+ temp[1] + "/matches/round/" + str(rodada), headers=headers)
        resp.encoding = 'UTF-8'
        ret.append(get_info_rodada(resp, de_para_siglas))
    
    ret = pd.concat(ret)
    ret.to_csv("links_sofa_score/sofa_score_links_EPL_"+temp[0].replace("/", "_")+".csv", sep=";")

### Pega informações dos jogos históricos

In [124]:
base = pd.read_csv("links_sofa_score/sofa_score_links_EPL_18_19.csv", sep=";")

In [53]:
s = requests.Session()
s.get('https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325')

<Response [200]>

In [130]:
list(reversed(sorted(os.listdir("links_sofa_score/"))))[1:]

['sofa_score_links_EPL_18_19.csv',
 'sofa_score_links_EPL_17_18.csv',
 'sofa_score_links_EPL_16_17.csv',
 'sofa_score_links_EPL_15_16.csv',
 'sofa_score_links_EPL_14_15.csv',
 'sofa_score_links_EPL_13_14.csv',
 'sofa_score_links_EPL_12_13.csv',
 'sofa_score_links_EPL_11_12.csv',
 'sofa_score_links_EPL_10_11.csv',
 'sofa_score_links_EPL_09_10.csv',
 'sofa_score_links_EPL_08_09.csv',
 'sofa_score_links_EPL_07_08.csv',
 'sofa_score_links_EPL_06_07.csv',
 'sofa_score_links_EPL_05_06.csv',
 'sofa_score_links_EPL_04_05.csv',
 'sofa_score_links_EPL_03_04.csv',
 'sofa_score_links_BR_2018.csv',
 'sofa_score_links_BR_2017.csv',
 'sofa_score_links_BR_2016.csv',
 'sofa_score_links_BR_2015.csv',
 'sofa_score_links_BR_2014.csv',
 'sofa_score_links_BR_2013.csv',
 'sofa_score_links_BR_2012.csv',
 'sofa_score_links_BR_2011.csv',
 'sofa_score_links_BR_2010.csv',
 'sofa_score_links_BR_2009.csv',
 'sofa_score_links_BR_2008.csv']

In [129]:
resp2.json()

{'error': {'code': 404, 'message': 'Statistics not found.'}}

In [150]:
de_para_siglas = pd.read_excel("de_para_siglas_epl.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

In [ ]:
for file in list(reversed(sorted(os.listdir("links_sofa_score/"))))[1:]:
    base = pd.read_csv("links_sofa_score/" + file, sep=";")
    
    resp_game = []
    resp_players = []
    
    name = file[-13:-4]
    
    i = 0
    for game_id in base.id:
        print(name + " Game #" + str(i), end="\r")
        resp = s.get("https://www.sofascore.com/event/" + str(game_id) + "/general/json", headers=headers)
        
        resp2 = s.get("https://www.sofascore.com/event/" + str(game_id) + "/statistics/players/json", headers=headers)
        resp2.encoding = 'UTF-8'
        try:
            players_df = pd.DataFrame(resp2.json()["players"])
        except KeyError:
            resp_game.append(pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0]).set_index("game"))
            continue
        
        resp_game.append(pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0]).set_index("game"))
        resp_players.append(get_per_player_data(players_df, de_para_siglas)) 
        i = i + 1
    
    ret_game = pd.concat(resp_game)
    ret_game.to_csv("EPL_data_sofa_score/game_data_"+ name +".csv", sep=";")
    
    ret_players = pd.concat(resp_players)
    ret_players.to_csv("EPL_data_sofa_score/players_data_"+ name +".csv", sep=";")